In [3]:
from dataclasses import dataclass
import os
from tempfile import TemporaryDirectory
from typing import List, Any, Dict

from shapely.geometry import box, mapping
import rasterio
from rasterio.warp import reproject, Resampling
import pyproj


ModuleNotFoundError: No module named 'osgeo'

In [5]:
from osgeo import gdal

from pystac_client import Client
import planetary_computer as pc

ModuleNotFoundError: No module named 'osgeo'

In [2]:
!pip install pyproj

     |████████████████████████████████| 7.5 MB 541 kB/s eta 0:00:01
You should consider upgrading via the '/Users/michaelka/Developer/ml/projects/p3env/bin/python -m pip install --upgrade pip' command.


In [6]:
# !pip install osgeo
!pip install gdal

     |████████████████████████████████| 747 kB 956 kB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_32cc283da1ad42abb3cb091dbb692683/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_32cc283da1ad42abb3cb091dbb692683/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-cyqw6v9f
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_32cc283da1ad42abb3cb091dbb692683/
    Complete output (118 lines):
    running egg_info
    creating /private/var/fold

     |████████████████████████████████| 746 kB 2.6 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_295e3224610b4bfd9c014d315ed1438b/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_295e3224610b4bfd9c014d315ed1438b/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-xudmx0ne
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_295e3224610b4bfd9c014d315ed1438b/
    Complete output (118 lines):
    running egg_info
    creating /private/var/fold

     |████████████████████████████████| 743 kB 2.3 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_541acaa2947a470389c9ac5b8a39f071/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_541acaa2947a470389c9ac5b8a39f071/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-4ie9u3mn
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_541acaa2947a470389c9ac5b8a39f071/
    Complete output (118 lines):
    running egg_info
    creating /private/var/fold

     |████████████████████████████████| 601 kB 2.5 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_7532e1c6ad4a401383542de5183ec1cf/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_7532e1c6ad4a401383542de5183ec1cf/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-2dkbd2qj
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_7532e1c6ad4a401383542de5183ec1cf/
    Complete output (118 lines):
    running egg_info
    creating /private/var/fold

     |████████████████████████████████| 601 kB 3.7 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_3c20cba7ab744260b2eed8084d190cc2/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_3c20cba7ab744260b2eed8084d190cc2/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-evd4mtv6
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_3c20cba7ab744260b2eed8084d190cc2/
    Complete output (118 lines):
    running egg_info
    creating /private/var/fold

...

     |████████████████████████████████| 477 kB 2.0 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_212ad405e7304e34a0cb758f078dc5f6/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_212ad405e7304e34a0cb758f078dc5f6/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-vd346qpn
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_212ad405e7304e34a0cb758f078dc5f6/
    Complete output (72 lines):
    running egg_info
    creating /private/var/folde

     |████████████████████████████████| 477 kB 3.1 MB/s eta 0:00:01     |████████████████████████▊       | 368 kB 3.1 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_4b031d26e6014d4798584c1843f33258/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_4b031d26e6014d4798584c1843f33258/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-w9rhal3k
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_4b031d26e6014d4798584c1843f33258/
    Complete outp

    writing /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-ptuprtu0/GDAL.egg-info/PKG-INFO
    writing dependency_links to /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-ptuprtu0/GDAL.egg-info/dependency_links.txt
    writing top-level names to /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-ptuprtu0/GDAL.egg-info/top_level.txt
    writing manifest file '/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-ptuprtu0/GDAL.egg-info/SOURCES.txt'
    Traceback (most recent call last):
      File "/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_d19bab4ceb9e40ecad4e2176a24ff0e6/setup.py", line 119, in fetch_config
        p = subprocess.Popen([command, args], stdout=subprocess.PIPE)
      File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/subprocess.py", line 854, in __init__
        self._execute_child(a

     |████████████████████████████████| 420 kB 1.7 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_c37d09f37d384e5d8c07ccce70ab49c7/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_c37d09f37d384e5d8c07ccce70ab49c7/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-pip-egg-info-5h5eyv1l
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_c37d09f37d384e5d8c07ccce70ab49c7/
    Complete output (52 lines):
    running egg_info
    creating /private/var/folde

Failed to build gdal
    Running setup.py install for gdal ... error
    ERROR: Command errored out with exit status 1:
     command: /Users/michaelka/Developer/ml/projects/p3env/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_5da8796c9bc24f7c97a2658487dfe52a/setup.py'"'"'; __file__='"'"'/private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-install-jxu8id2i/gdal_5da8796c9bc24f7c97a2658487dfe52a/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv300000gn/T/pip-record-v890d8_3/install-record.txt --single-version-externally-managed --compile --install-headers /Users/michaelka/Developer/ml/projects/p3env/include/site/python3.8/gdal
         cwd: /private/var/folders/pb/x_y9yw2j27bc72jcwrndmv3000

In [9]:
!sudo apt-get install libgdal-dev

Password:
sudo: a password is required


In [8]:
!qqww

zsh:1: command not found: qqww
